In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.ndimage import gaussian_filter
from brainglobe_utils.image.heatmap import heatmap_from_points
import tifffile

In [59]:
allow_pickle=True

# Define paths to subjects (Each subject has its own atlas.points and atlas_regions.npy)

group1_subjects = [
    {"points": Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\026\brainreg_gr_4\points\atlas.points"),
    "regions": Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\026\brainreg_gr_4\registration\registered_atlas.tiff")},
    {"points": Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\833\brainreg_gr_3\points\atlas.points"),
    "regions": Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\833\brainreg_gr_3\registration\registered_atlas.tiff")},
    {"points": Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\987\brainreg_1\points\atlas.points"),
    "regions": Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\987\brainreg_1\registration\registered_atlas.tiff")},
    {"points": Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\cFos61\brainreg_trained_trained_74model_changedparam_4\points\atlas.points"),
    "regions": Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\cFos61\brainreg_trained_trained_74model_changedparam_4\registration\registered_atlas.tiff")},
    {"points": Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\cFos74\brainreg_trained_BB_10\points\atlas.points"),
     "regions": Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\cFos74\brainreg_trained_BB_10\registration\registered_atlas.tiff")}
]

group2_subjects = [
    {"points": Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\Control\834\brainreg_gr_mixed_4\points\atlas.points"),
    "regions": Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\Control\834\brainreg_gr_mixed_4\registration\registered_atlas.tiff")},
    {"points": Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\Control\838\brainreg_gr_4\points\atlas.points"),
    "regions": Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\Control\838\brainreg_gr_4\registration\registered_atlas.tiff")},
    {"points": Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\Control\981\brainreg_3\points\atlas.points"),
    "regions": Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\Control\981\brainreg_3\registration\registered_atlas.tiff")},
    {"points": Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\Control\cFos60\brainreg_trained_after74_6\points\atlas.points"),
    "regions": Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\Control\cFos60\brainreg_trained_after74_6\registration\registered_atlas.tiff")}
]

# Function to compute mean density per brain region for a list of subjects
def compute_density(subjects):
    region_density = {}

    for subject in subjects:
        # Load subject-specific atlas and cell points
        points = pd.read_hdf(subject["points"], key="df").values
        atlas_labels = np.load(subject["regions"])  # Load subject-specific region mapping

        unique_regions = np.unique(atlas_labels)  # Get all brain regions in this subject's atlas

        # Compute histogram (cell count per voxel)
        heatmap, _ = np.histogramdd(points, bins=atlas_labels.shape)

        # Compute density per region
        for region in unique_regions:
            region_mask = atlas_labels == region
            mean_density = heatmap[region_mask].sum() / region_mask.sum()  # Normalize by voxel count

            if region not in region_density:
                region_density[region] = []
            region_density[region].append(mean_density)

    # Compute the mean density across all subjects in the group
    mean_densities = {region: np.mean(region_density[region]) for region in region_density}
    return mean_densities

# Compute mean densities for each group (makes background bright?)
group1_densities = compute_density(group1_subjects)
group2_densities = compute_density(group2_subjects)

# Create brain-wide density maps
def create_density_map(region_densities, reference_atlas):
    density_map = np.zeros_like(reference_atlas, dtype=float)
    for region, value in region_densities.items():
        density_map[reference_atlas == region] = value
    return density_map

# Load a reference atlas from any subject (assuming all atlases have the same shape?)
reference_atlas = np.load(group1_subjects[0]["regions"])  

group1_density_map = create_density_map(group1_densities, reference_atlas)
group2_density_map = create_density_map(group2_densities, reference_atlas)

# Apply Gaussian smoothing
smoothed_group1 = gaussian_filter(group1_density_map, sigma=5)
smoothed_group2 = gaussian_filter(group2_density_map, sigma=5)

# Generate heatmaps for each group
heatmap_from_points(
    group1_density_map, 
    image_resolution=25,
    image_shape=reference_atlas.shape,
    output_filename="group1_density_heatmap.tiff",
    smoothing=50
)

heatmap_from_points(
    group2_density_map, 
    image_resolution=25,
    image_shape=reference_atlas.shape,
    output_filename="group2_density_heatmap.tiff",
    smoothing=50
)


ValueError: Cannot load file containing pickled data when allow_pickle=False

In [4]:
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.ndimage import gaussian_filter
from brainglobe_utils.image.heatmap import heatmap_from_points
import tifffile

In [5]:

##############################################################################
# 1. Define Subjects (each subject has a path to 'atlas.points')
##############################################################################
group1_points_files = [
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\026\brainreg_gr_4\points\atlas.points"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\833\brainreg_gr_3\points\atlas.points"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\987\brainreg_1\points\atlas.points"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\cFos61\brainreg_trained_trained_74model_changedparam_4\points\atlas.points"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\cFos74\brainreg_trained_BB_10\points\atlas.points"),
]

group1_atlas_files = [
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\026\brainreg_gr_4\registration\registered_atlas.tiff"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\833\brainreg_gr_3\registration\registered_atlas.tiff"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\987\brainreg_1\registration\registered_atlas.tiff"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\cFos61\brainreg_trained_trained_74model_changedparam_4\registration\registered_atlas.tiff"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\cFos74\brainreg_trained_BB_10\registration\registered_atlas.tiff"),
]

group2_points_files = [
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\Control\834\brainreg_gr_mixed_4\points\atlas.points"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\Control\838\brainreg_gr_4\points\atlas.points"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\Control\981\brainreg_3\points\atlas.points"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\Control\cFos60\brainreg_trained_after74_6\points\atlas.points")
]

group2_atlas_files = [
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\Control\834\brainreg_gr_mixed_4\registration\registered_atlas.tiff"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\Control\838\brainreg_gr_4\registration\registered_atlas.tiff"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\Control\981\brainreg_3\registration\registered_atlas.tiff"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\Control\cFos60\brainreg_trained_after74_6\registration\registered_atlas.tiff")
]

In [55]:

##############################################################################
# 1. Define Subjects (each subject has a path to 'atlas.points')
##############################################################################
group1_subjects = [
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\026\brainreg_gr_4\points\atlas.points"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\833\brainreg_gr_3\points\atlas.points"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\987\brainreg_1\points\atlas.points"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\cFos61\brainreg_trained_trained_74model_changedparam_4\points\atlas.points"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\cFos74\brainreg_trained_BB_10\points\atlas.points"),
]


group2_subjects = [
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\Control\834\brainreg_gr_mixed_4\points\atlas.points"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\Control\838\brainreg_gr_4\points\atlas.points"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\Control\981\brainreg_3\points\atlas.points"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\Control\cFos60\brainreg_trained_after74_6\points\atlas.points")
]



In [56]:
##############################################################################
# 2. Determine the Atlas Shape (bins) & Load Sample Points
##############################################################################
atlas_shape = (1320, 903, 1173)  # Adjust

In [49]:
##############################################################################
# 3. Create Helper Functions
##############################################################################

def load_points(points_file):
    """Load cell coordinates (Nx3) from an HDF5 atlas.points file."""
    df = pd.read_hdf(points_file, key="df")
    return df.values  # Nx3 array

def subject_hist(points_file, atlas_mask, atlas_shape):
    """
    Convert a subject’s Nx3 points into a 3D masked histogram.
    Non-brain regions (mask=0) will be set to NaN.
    """
    coords = load_points(points_file)
    x_bins = np.arange(0, atlas_shape[0] + 1)
    y_bins = np.arange(0, atlas_shape[1] + 1)
    z_bins = np.arange(0, atlas_shape[2] + 1)
    
    hist, _ = np.histogramdd(coords, bins=[x_bins, y_bins, z_bins])
    
    # Apply the atlas mask (set non-brain regions to NaN)
    masked_hist = hist * atlas_mask
    masked_hist[atlas_mask == 0] = np.nan
    
    return masked_hist

def load_group_voxels(points_files, atlas_files, atlas_shape):
    """
    Load histograms for all subjects in a group, applying the specific atlas mask to each.
    """
    all_hists = []
    for points_file, atlas_mask_file in zip(points_files, atlas_files):
        
        # Load the specific atlas mask
        atlas_mask = tifffile.imread(str(atlas_mask_file))
        atlas_mask = (atlas_mask > 0).astype(np.uint8)  # Ensure binary mask
        
        # Rescale atlas mask if needed
        if atlas_mask.shape != atlas_shape:
            print(f"Rescaling atlas mask from {atlas_mask.shape} to {atlas_shape}")
            scaling_factors = np.array(atlas_shape) / np.array(atlas_mask.shape)
            atlas_mask = zoom(atlas_mask, scaling_factors, order=0)  # Nearest-neighbor
        
        # Load and mask the subject's cell data
        masked_hist = subject_hist(points_file, atlas_mask, atlas_shape)
        all_hists.append(masked_hist)
    
    return np.array(all_hists)  # shape: (num_subjects, X, Y, Z)


In [64]:
##############################################################################
# 3.1 Load and Mask Histograms for Both Groups
##############################################################################

group1_hists = load_group_voxels(group1_points_files, group1_atlas_files, atlas_shape)
group2_hists = load_group_voxels(group2_points_files, group2_atlas_files, atlas_shape)

# Calculate the mean and std only in brain regions (ignoring NaNs)
group1_mean = np.nanmean(group1_hists, axis=0)
group2_mean = np.nanmean(group2_hists, axis=0)

group1_std = np.nanstd(group1_hists, axis=0)
group2_std = np.nanstd(group2_hists, axis=0)

Rescaling atlas mask from (1400, 838, 1254) to (1399, 903, 1173)
Rescaling atlas mask from (1400, 827, 1204) to (1399, 903, 1173)
Rescaling atlas mask from (1412, 1035, 1203) to (1399, 903, 1173)
Rescaling atlas mask from (1288, 903, 1173) to (1399, 903, 1173)
Rescaling atlas mask from (1288, 841, 1090) to (1399, 903, 1173)
Rescaling atlas mask from (1436, 814, 1136) to (1399, 903, 1173)
Rescaling atlas mask from (1436, 859, 1113) to (1399, 903, 1173)
Rescaling atlas mask from (1460, 723, 884) to (1399, 903, 1173)
Rescaling atlas mask from (1288, 958, 1126) to (1399, 903, 1173)


C:\Users\admin\AppData\Local\Temp\ipykernel_29924\640698736.py:9: RuntimeWarning: Mean of empty slice
  group1_mean = np.nanmean(group1_hists, axis=0)
C:\Users\admin\AppData\Local\Temp\ipykernel_29924\640698736.py:10: RuntimeWarning: Mean of empty slice
  group2_mean = np.nanmean(group2_hists, axis=0)
c:\Users\admin\anaconda3\envs\brainglobe-test\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [65]:
pooled_std = np.sqrt((group1_std**2 + group2_std**2) / 2)
cohens_d = (group1_mean - group2_mean) / (pooled_std + 1e-9)  # Avoid /0

# Replace NaNs with 0 (makes bright background?)
cohens_d[np.isnan(cohens_d)] = 0

In [66]:
output_file = "masked_cohens_d_3d.tiff"
tifffile.imwrite(output_file, cohens_d.astype(np.float32))
print(f"Masked Cohen's d heatmap saved to {output_file}")

Masked Cohen's d heatmap saved to masked_cohens_d_3d.tiff


In [57]:
def load_points(points_file):
    """Load cell coordinates (Nx3) from an HDF5 atlas.points file."""
    df = pd.read_hdf(points_file, key="df")
    return df.values  # Nx3 array

def subject_hist(points_file, atlas_shape):
    """
    Convert a subject’s Nx3 points into a 3D histogram with shape=atlas_shape.
    Bins = [0..atlas_shape[0]], [0..atlas_shape[1]], [0..atlas_shape[2]].
    """
    coords = load_points(points_file)
    x_bins = np.arange(0, atlas_shape[0] + 1)
    y_bins = np.arange(0, atlas_shape[1] + 1)
    z_bins = np.arange(0, atlas_shape[2] + 1)
    hist, edges = np.histogramdd(coords, bins=[x_bins, y_bins, z_bins])
    return hist

def load_group_voxels(subject_list, atlas_shape):
    """
    Return a list of 3D histograms (one per subject).
    """
    all_hists = []
    for subj in subject_list:
        h = subject_hist(subj, atlas_shape)
        all_hists.append(h)
    return np.array(all_hists)  # shape: (num_subjects, X, Y, Z)


In [51]:
print(group1_subjects)

[WindowsPath('Y:/public/projects/AnAl_20240405_Neuromod_PE/brainsaw/PE_mapping/PE/block2/026/brainreg_gr_4/points/atlas.points'), WindowsPath('Y:/public/projects/AnAl_20240405_Neuromod_PE/brainsaw/PE_mapping/PE/block2/833/brainreg_gr_3/points/atlas.points'), WindowsPath('Y:/public/projects/AnAl_20240405_Neuromod_PE/brainsaw/PE_mapping/PE/block2/987/brainreg_1/points/atlas.points'), WindowsPath('Y:/public/projects/AnAl_20240405_Neuromod_PE/brainsaw/PE_mapping/PE/block2/cFos61/brainreg_trained_trained_74model_changedparam_4/points/atlas.points'), WindowsPath('Y:/public/projects/AnAl_20240405_Neuromod_PE/brainsaw/PE_mapping/PE/block2/cFos74/brainreg_trained_BB_10/points/atlas.points')]


In [58]:
group1_hists = load_group_voxels(group1_subjects, atlas_shape)  # shape (N1, X, Y, Z)
group2_hists = load_group_voxels(group2_subjects, atlas_shape)  # shape (N2, X, Y, Z)

# Mean voxel-wise cell counts
group1_mean = np.mean(group1_hists, axis=0)  # shape (X, Y, Z)
group2_mean = np.mean(group2_hists, axis=0)

# Std voxel-wise
group1_std = np.std(group1_hists, axis=0)
group2_std = np.std(group2_hists, axis=0)

# (Optional) Convert to density
#group1_mean /= group1_mean.sum()
#group2_mean /= group2_mean.sum()


MemoryError: Unable to allocate 52.1 GiB for an array with shape (5, 1320, 903, 1173) and data type float64

In [54]:
pooled_std = np.sqrt((group1_std**2 + group2_std**2) / 2)

cohens_d = (group1_mean - group2_mean) / (pooled_std + 1e-9)  # avoid /0
cohens_d[np.isnan(cohens_d)] = 0  # replace any NaNs


In [33]:
diff_meanden = (group1_mean - group2_mean)
print(diff_meanden)


[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


In [31]:
import tifffile

# Smaller sigma => less blur. sigma=1 => minimal smoothing, sigma=0 => none
smoothed_cohens_d = gaussian_filter(cohens_d, sigma=10)

# Save to 3D TIFF
tifffile.imwrite("cohens_d_blur10.tiff", smoothed_cohens_d.astype(np.float32))

print("Saved minimal-blur Cohen’s d volume to 'cohens_d_miniblur.tiff'.")


Saved minimal-blur Cohen’s d volume to 'cohens_d_miniblur.tiff'.


In [34]:
import tifffile

# Smaller sigma => less blur. sigma=1 => minimal smoothing, sigma=0 => none
smoothed_diff_mean = gaussian_filter(diff_meanden, sigma=10)

# Save to 3D TIFF
tifffile.imwrite("diff_meanden_blur10.tiff", smoothed_diff_mean.astype(np.float32))

#print("Saved minimal-blur Cohen’s d volume to 'cohens_d_miniblur.tiff'.")

save the raw data as a CSV

In [53]:

atlas_mask_file = "annotation_10.tiff"
atlas_mask = tifffile.imread(atlas_mask_file)

# Extract unique region IDs from the annotation file (excluding 0 for background)
unique_regions = np.unique(atlas_mask)
unique_regions = unique_regions[unique_regions > 0]

# Prepare data for CSV export
region_data = []
for region_id in unique_regions:
    region_mask = (atlas_mask == region_id)
    
    # Calculate the mean Cohen's d for each region
    region_values = cohens_d[region_mask]
    smoothed_values = smoothed_cohens_d[region_mask]
    
    mean_cohens_d = np.nanmean(region_values)
    mean_smoothed_cohens_d = np.nanmean(smoothed_values)
    
    region_data.append({
        'Region ID': region_id,
        "Raw Cohen's d": mean_cohens_d,
        "Smoothed Cohen's d": mean_smoothed_cohens_d
    })

# Convert to a DataFrame
region_df = pd.DataFrame(region_data)

# Save the DataFrame to a CSV file
region_df.to_csv('cohens_d_by_region.csv', index=False)
print("Saved raw and smoothed Cohen's d data to 'cohens_d_by_region.csv'.")


print(region_df.head())


IndexError: boolean index did not match indexed array along dimension 0; dimension is 1399 but corresponding boolean dimension is 1320

In [ ]:
region_df.to_csv('cohens_d_by_region.csv', index=False)
print("Saved raw data to 'cohens_d_by_region.csv'")

In [46]:
import napari
import numpy as np
from skimage import io, filters

# Load the image stack
image_stack = io.imread("Y:/public/projects/AnAl_20240405_Neuromod_PE/brainsaw/PE_mapping/figures/cohens_d_blur5.tiff")

# Apply a threshold to remove the background
threshold_value = filters.threshold_otsu(image_stack)
foreground_mask = image_stack > threshold_value

# Apply the mask to keep only the brain
foreground_stack = image_stack * foreground_mask

output_path = r"cohens_d_blur5_processed.tiff"

# Save the image stack as a TIFF file
io.imsave(output_path, foreground_stack.astype(np.uint16))

C:\Users\admin\AppData\Local\Temp\ipykernel_29924\2048137086.py:18: UserWarning: cohens_d_blur5_processed.tiff is a low contrast image
  io.imsave(output_path, foreground_stack.astype(np.uint16))


In [8]:
##############################################################################
# 4. Compute Mean Densities for Each Group
##############################################################################
group1_mean_hist, group1_all = mean_density_across_subjects(group1_subjects, atlas_shape)
group2_mean_hist, group2_all = mean_density_across_subjects(group2_subjects, atlas_shape)

# Normalize by total cell count per group
group1_density = group1_mean_hist / group1_mean_hist.sum()
group2_density = group2_mean_hist / group2_mean_hist.sum()

NameError: name 'mean_density_across_subjects' is not defined

In [26]:
##############################################################################
# 5. Compute Cohen’s d for Each Voxel
##############################################################################

group1_std = np.std(group1_all, axis=0)  # shape: (X, Y, Z)
group2_std = np.std(group2_all, axis=0)

# Compute pooled std (unbiased version):
pooled_std = np.sqrt((group1_std**2 + group2_std**2) / 2)

# Cohen’s d: (mu1 - mu2) / pooled_std
cohens_d = (group1_density - group2_density) / (pooled_std + 1e-9)  # to avoid /0
# Replace NaNs
cohens_d[np.isnan(cohens_d)] = 0

In [27]:
##############################################################################
# 6. (Optional) Smooth the 3D Arrays
##############################################################################
smoothed_group1 = gaussian_filter(group1_density, sigma=2)
smoothed_group2 = gaussian_filter(group2_density, sigma=2)
smoothed_cohens_d = gaussian_filter(cohens_d, sigma=2)

In [19]:
##############################################################################
# 7. Save or Visualize the Heatmaps
##############################################################################
# Option A: Using 'heatmap_from_points()' with “fake” points
# 'heatmap_from_points()' expects Nx3 points array. 
#  convert 3D array to points, weighting each voxel by the density or Cohen’s d value (is this correct?).

def array_to_points_3d(volume_3d):
    """
    Convert a 3D volume into a list of points, 
    repeating each voxel coordinate according to the voxel value (if integer).
    
    But for continuous data (like densities, we might store them as 'weights').
    For 'heatmap_from_points()', we can pass 'points' & 'weights' with histogramdd too.
    """
    points = []
    for x in range(volume_3d.shape[0]):
        for y in range(volume_3d.shape[1]):
            for z in range(volume_3d.shape[2]):
                val = volume_3d[x, y, z]
                # If val is small or zero, skip
                if val > 0:
                    points.append([x, y, z])
    return np.array(points, dtype=float)

# Convert the smoothed Cohen’s d array to "points"
cohens_d_points = array_to_points_3d(smoothed_cohens_d)

# Then call 'heatmap_from_points()'
heatmap_from_points(
    points=cohens_d_points,
    image_resolution=1.0,  # 1 voxel = 1 unit (one cell?)
    image_shape=atlas_shape,
    output_filename="cohens_d_heatmap.tiff",
    smoothing=0  #  already smoothed in 3D
)

# Option B: Save 'smoothed_cohens_d' to .npy:
# np.save("cohens_d_3d.npy", smoothed_cohens_d)

Done computing mean densities and Cohen's d heatmap.


In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from brainglobe_utils.image.heatmap import heatmap_from_points

group1_subjects = [
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\026\brainreg_gr_4\points\atlas.points"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\833\brainreg_gr_3\points\atlas.points"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\987\brainreg_1\points\atlas.points"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\cFos61\brainreg_trained_trained_74model_changedparam_4\points\atlas.points"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\PE\block2\cFos74\brainreg_trained_BB_10\points\atlas.points"),
]

# Combine all subject points
all_group1_points = []
for subj in group1_subjects:
    # Load downsampled (x,y,z)
    df = pd.read_hdf(subj, key="df")
    points_array = df.values  # shape: (N, 3)
    all_group1_points.append(points_array)

# Stack
all_group1_points = np.vstack(all_group1_points)
print("Group 1 total cells:", all_group1_points.shape[0])
print( all_group1_points)

Group 1 total cells: 6964342
[[1330  593  445]
 [1348  425  576]
 [1334  451  585]
 ...
 [  76  600  349]
 [ 139  580  720]
 [1231  478  660]]


In [31]:
import numpy as np
import pandas as pd
from pathlib import Path
from brainglobe_utils.image.heatmap import heatmap_from_points

group2_subjects = [
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\Control\834\brainreg_gr_mixed_4\points\atlas.points"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\Control\838\brainreg_gr_4\points\atlas.points"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\Control\981\brainreg_3\points\atlas.points"),
    Path(r"Y:\public\projects\AnAl_20240405_Neuromod_PE\brainsaw\PE_mapping\Control\cFos60\brainreg_trained_after74_6\points\atlas.points")
]

# Combine all subject points
all_group2_points = []
for subj in group2_subjects:
    # Load downsampled (x,y,z)
    df = pd.read_hdf(subj, key="df")
    points_array = df.values  # shape: (N, 3)
    all_group2_points.append(points_array)

# Stack
all_group2_points = np.vstack(all_group2_points)
print("Group 2 total cells:", all_group2_points.shape[0])


Group 2 total cells: 2005181


In [3]:
image_shape = (1399, 903, 1173)  # Adjust to match  downsampled atlas (get the standard space values 1320 ... ....)
voxel_resolution = 10          # Allen Mouse Brain 10 µm

heatmap_from_points(
    points=all_group1_points,
    image_resolution=voxel_resolution,
    image_shape=image_shape,
    output_filename="group1_mean_density.tiff",
    smoothing=100,        # sigma=100/25=4 voxels
    bin_sizes=(1,1,1),    
    mask_image=None,      # or load a mask
)
print("Saved group1_mean_density.tiff for Group 1.")


Saved group1_mean_density.tiff for Group 1.


In [34]:

heatmap_from_points(
    points=all_group2_points,
    image_resolution=voxel_resolution,
    image_shape=image_shape,
    output_filename="group2_mean_density.tiff",
    smoothing=100,
    bin_sizes=(1,1,1),   
    mask_image=None,  
)
print("Saved group2_mean_density.tiff for Group 2.")


Saved group2_mean_density.tiff for Group 2.


In [47]:
import numpy as np
import tifffile
from scipy.ndimage import zoom

# Load the registered atlas file from registartion folder

import numpy as np
import tifffile
from scipy.ndimage import zoom

# Load the Cohen's d heatmap
cohens_d_file = "cohens_d_blur5.tiff"
cohens_d_data = tifffile.imread(cohens_d_file)

# Load the atlas mask 
atlas_mask_file = "Y:/public/projects/AnAl_20240405_Neuromod_PE/brainsaw/PE_mapping/PE/block2/833/brainreg_gr_3/registration/registered_atlas.tiff"

atlas_mask = tifffile.imread(atlas_mask_file)

# Ensure the mask is binary
atlas_mask = (atlas_mask > 0).astype(np.uint8)

# Check if shapes match
# rescale the atlas mask
if atlas_mask.shape != cohens_d_data.shape:
    print(f"Rescaling atlas mask from {atlas_mask.shape} to {cohens_d_data.shape}")
    scaling_factors = np.array(cohens_d_data.shape) / np.array(atlas_mask.shape)
    atlas_mask = zoom(atlas_mask, scaling_factors, order=0)

# Apply the mask to the Cohen's d data
masked_cohens_d = cohens_d_data * atlas_mask

# Background (non-brain) regions to NaN (makes background bright?)
masked_cohens_d[atlas_mask == 0] = np.nan

# Save the masked data as TIFF
output_file = "masked_cohens_d_3d.tiff"
tifffile.imwrite(output_file, masked_cohens_d.astype(np.float32))

print(f"Masked Cohen's d heatmap saved to {output_file}")



Masked Cohen's d heatmap saved to masked_cohens_d_3d.tiff
